##  NLP- Bert Sentence Encoding



In [3]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# bert_model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

bert_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 1.3MB 17.3MB/s 
     |████████████████████████████████| 890kB 43.8MB/s 
     |████████████████████████████████| 2.9MB 50.2MB/s 
     |████████████████████████████████| 1.1MB 60.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101035 sha256=8964feff595fcbb38173acecf34b7f85636027d9bd18a1ba5936883e71d6a3f9
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c625673eb9fd7b422d7828a42f541ede4cef1448a116a4d81d8a3355676d8b63
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


100%|██████████| 1.01G/1.01G [01:18<00:00, 13.0MB/s]


In [ ]:
# # !pip install pytorch_transformers
# from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig

# # tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')


# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

# text = 'I liked that book very much!'
# tokenized_text = tokenizer.tokenize(text)
# print(tokenized_text)

# text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
# print('text ids:', text_ids)


## Preparing the dataset 

In [ ]:
# #student_book (student, book)

# student_book = pd.read_csv('student_book.csv')
# student_book.head() 
# student_book.shape #(487, 983)
#book text embedding
# book= pd.read_csv('/content/drive/MyDrive/명륜이의 서재/DATA/1206_data/재영 부분/book_100000_196940(wo null).csv') #this is the data we crawled
# text_book = pd.read_csv('book.csv')
# text_book.shape 
# # book.head()
# text_book.head()

In [ ]:
# #match the title and id 
# book = book[['book_id', 'subject']]
# book = book.drop_duplicates('book_id')
# book.head()#(66660, 2)
# text_book['title'] = book[:1000]['subject']
# text_book.head()
# text_book =text_book.drop_duplicates('book_id')
# text_book =text_book.reset_index(drop=True)
# text_book.head()
# text_book.iloc[815:]
# text_book= text_book.sort_values('book_id')
# text_book.head()
#needs reset of index 
# sentence_embeddings = bert_model.encode(text_book['title'])
# sentence_embeddings.shape #(6, 768) #this is the text embeddings for each book

#genre embedding 
# text_book =text_book.drop_duplicates('book_id')
# text_book= text_book.sort_values('book_id')
# text_book =text_book.reset_index(drop=True)
# genre =text_book[['book_id','ddc-sort']]
# genre.shape #(983, 2)
# genre = genre.set_index('book_id')
# genre.head()
# # genre['ddc-sort'].value_counts()
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder()
# enc.fit(genre[['ddc-sort']])
# genre_initial= enc.transform(genre[['ddc-sort']]).toarray()
# genre_initial

## Start From Here! 


In [4]:
student = pd.read_csv('student-3.csv')
book = pd.read_csv('bookexample.csv')
book.head()

NameError: ignored

### 1. Find the longest/average length for the books read for each student


In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline

sequences = student['booklist']
max_len = 55

print('문장의 최대 길이 :',max(len(l) for l in sequences)) #10 
print('문장의 평균 길이 :',sum(map(len, sequences))/len(sequences))
plt.hist([len(s) for s in sequences], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

### 2. Change the y value to a new df where all the values are one-hot-encoded 


In [ ]:
from tensorflow.keras.utils import to_categorical
y = student[['pred']] #(252,)
train_y = pd.DataFrame(index=y.index,columns=range(0,983))
train_y.fillna(0, inplace=True)

for i in range(len(y)):
  # print(y.iloc[i,0])
  train_y.iloc[i,int(y.iloc[i,0])] = 1 


train_y.head()
# train_y.shape #(252, 983)

### 3. Make the book list into a list of lists + padd according to max_value

In [5]:
books = student['booklist'].tolist()
type(books[0])
books
new=[]
from ast import literal_eval
for x in books:
  x = literal_eval(x)
  new.append(x)

new[:10]

NameError: ignored

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_new = pad_sequences(new, maxlen = max_len)
pad_new.shape #(252, 55)

(252, 55)

### 4. Embedd the titles using bert - get the embeddings 

In [ ]:

# book.shape #(983, 5)
# book.head()
# sentence_embeddings = bert_model.encode(book['subject'])
# sentence_embeddings.shape #(983, 768)
sentence_embeddings[0]

array([-4.77335542e-01,  3.71796191e-01,  4.22544748e-01,  7.59998620e-01,
       -3.85853112e-01, -7.17986748e-02,  4.27637130e-01, -1.43503487e-01,
        2.13111416e-01, -4.03409362e-01,  1.65444076e-01,  3.80740166e-01,
        1.42300367e-01, -4.24716651e-01, -5.30597329e-01,  7.15721846e-01,
       -7.49361217e-01, -4.26458329e-01,  1.04344428e-01, -4.20641869e-01,
        2.94024169e-01, -4.12547678e-01,  1.95334524e-01,  1.18603818e-01,
        7.17893004e-01,  5.93436837e-01, -5.44176959e-02, -3.98434252e-01,
       -3.02409202e-01,  4.67815667e-01,  2.02830911e-01,  2.48822168e-01,
       -3.70882332e-01, -3.65570247e-01, -4.82681423e-01, -8.54683295e-03,
        4.74348456e-01,  5.52435398e-01,  3.30556184e-01,  1.55728921e-01,
        3.39899302e-01,  4.12307501e-01,  3.81828755e-01, -4.35026467e-01,
       -1.36849970e-01,  8.58883485e-02, -1.17381728e+00, -9.22385678e-02,
        7.37570301e-02, -3.23347092e-01, -5.13782680e-01,  2.82991510e-02,
        1.95974156e-01,  

### 5. Create the deep learning model using BERT 

In [ ]:
# sentence_embeddings[0]
#student_book

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten,BatchNormalization, Dropout, Reshape

model = Sequential()        #input = number of books #output dim = embedding shape  #input_length : max_len since sentence
genre_embeddings  = Embedding(input_dim = sentence_embeddings.shape[0], #983  
                              output_dim=sentence_embeddings.shape[1], #768
                              input_length= max_len, 
                              weights = [sentence_embeddings],trainable=True)
model.add(genre_embeddings)
model.add( Flatten() )
model.add(Dense(900, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(990, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(983, activation='softmax')) 
model.compile(optimizer='adam', loss='categorical_crossentropy',	metrics=["accuracy"])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 55, 768)           754944    
_________________________________________________________________
flatten_1 (Flatten)          (None, 42240)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 900)               38016900  
_________________________________________________________________
batch_normalization_2 (Batch (None, 900)               3600      
_________________________________________________________________
dropout_2 (Dropout)          (None, 900)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 990)               891990    
_________________________________________________________________
batch_normalization_3 (Batch (None, 990)              

In [ ]:
history = model.fit(pad_new,
                    train_y,
                    epochs=100,
                    batch_size=5)

Epoch 1/100
51/51 [==============================] - 2s 19ms/step - loss: 7.9187 - accuracy: 0.0000e+00
Epoch 2/100
51/51 [==============================] - 1s 19ms/step - loss: 7.1037 - accuracy: 0.0045
Epoch 3/100
51/51 [==============================] - 1s 19ms/step - loss: 6.2974 - accuracy: 0.0400
Epoch 4/100
51/51 [==============================] - 1s 19ms/step - loss: 5.6226 - accuracy: 0.0618
Epoch 5/100
51/51 [==============================] - 1s 19ms/step - loss: 5.1674 - accuracy: 0.0938
Epoch 6/100
51/51 [==============================] - 1s 22ms/step - loss: 4.2286 - accuracy: 0.1895
Epoch 7/100
51/51 [==============================] - 1s 19ms/step - loss: 3.8633 - accuracy: 0.2289
Epoch 8/100
51/51 [==============================] - 1s 19ms/step - loss: 3.3284 - accuracy: 0.2957
Epoch 9/100
51/51 [==============================] - 1s 19ms/step - loss: 3.0448 - accuracy: 0.3762
Epoch 10/100
51/51 [==============================] - 1s 19ms/step - loss: 2.3755 - accuracy: 0.

### 6. Test out the model (this case I used the training values, so its almost 100%)


In [ ]:
# model.evaluate(student_book.iloc[0], y.iloc[0])
yprob= model.predict(pad_new[:2])
yprob.argmax(axis=-1) #
model.predict_classes(pad_new[:2])
# yprob.shape #(2, 983)
# yprob

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([460, 835])

In [ ]:
train_y.iloc[:2, 460:] #460 predict 
train_y.iloc[:2, 830:] #835 predict

,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,...,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## cf) for visualization we need to make the embeddings with this formula


In [ ]:
#for visualization 
# Convert NumPy array of embedding into data frame
embedding_df = pd.DataFrame(sentence_embeddings)

# Save dataframe as as TSV file without any index and header
embedding_df.to_csv('output.tsv', sep='\t', index=None, header=None)
book.to_csv('metadata.tsv', index=False, sep='\t')